## WikiData PDF loading

Мы взяли все сущности, которые являются `musical group` ([Q215380](https://www.wikidata.org/wiki/Q215380)) (либо каким-то их подклассом), а также всех людей, у которых в графе `occupation` указано `singer` ([Q177220](https://www.wikidata.org/wiki/Q177220))

В итоге получилось собрать данные для 76104 групп и 87430 певцов.
В рамках этой работы они все будут считаться исполнителями.

Для каждого автора мы собрали следующую информацию: название на английском языке, список характерных жанров ([P136](https://www.wikidata.org/wiki/Property:P136)), страна происхождения ([P495](https://www.wikidata.org/wiki/Property:P495)), источники вдохновения ([P737](https://www.wikidata.org/wiki/Property:P737)), список наград ([P166](https://www.wikidata.org/wiki/Property:P166)). Название использовалось для сопоставления исполнителей с данными датасета Last.fm, а остальные связи (588521 отношений) использовались для построения графа знаний.

In [247]:
import json
from collections import Counter

In [248]:
datas = dict()
authors = dict()

In [249]:
def parse_data(p, relations, authors):
    with open(p, 'r', encoding='utf-8') as f:
        data = json.load(f)
    for item in data:
        if not item['label']:
            continue
        authors[item['label']] = item['item']
        datas[item['label']] = item

In [250]:
parse_data('wikidata/strange_groups.json', datas, authors)
parse_data('wikidata/groups_0.json', datas, authors)
parse_data('wikidata/groups_1.json', datas, authors)
parse_data('wikidata/singers_0.json', datas, authors)
parse_data('wikidata/singers_1.json', datas, authors)

## Transform Last.fm dataset
Оригинальная статья: [The LFM-1b Dataset for Music Retrieval and Recommendation](https://www.jku.at/fileadmin/gruppen/173/Research/schedl_icmr_2016.pdf)

Сам датасет можно найти [здесь](http://www.cp.jku.at/datasets/LFM-1b/)

Датасет состоит из 1 088 161 692 прослушиваний пользователями треков с Last.fm. 

Количество пользователей:    120 323

Количество треков:        32 291 134

Количество альбомов:      15 991 038

Количество исполнителей:   3 190 371

In [355]:
artist2wd = {}

with open('LFM-1b/LFM-1b_artists.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) != 2:
            continue
        artist_id, name = parts
        if name in authors:
            artist2wd[int(artist_id)] = authors[name]

In [356]:
print(f'Для {len(artist2wd)} исполнителей удалось найти соответствие сущностей из wikidata и id-шниками из датасета Last.fm')

Для 91001 исполнителей удалось найти соответствие сущностей из wikidata и id-шниками из датасета Last.fm


In [357]:
user_cnt = Counter()
artist_cnt = Counter()
song_cnt = Counter()
interactions = []

In [358]:
with open('LFM-1b/LFM-1b_LEs.txt', 'r', encoding='utf-8') as f:
    for line in f:
        if random.random() > 0.00125:
            continue
        
        parts = line.strip().split('\t')
        
        user_id = int(parts[0])
        artist_id = int(parts[1])
        albom_id = int(parts[2])
        song_id = int(parts[3])
        
        if artist_id not in artist2wd:
            continue
        
        interactions.append((user_id, artist_id, albom_id, song_id))
        
        user_cnt[user_id] += 1
        artist_cnt[artist_id] += 1
        song_cnt[song_id] += 1

In [359]:
interactions = list([(user_id, artist_id, albom_id, song_id) for user_id, artist_id, albom_id, song_id in set(interactions) 
                     if user_cnt[user_id] > 5 and song_cnt[song_id] > 10])

In [360]:
len(interactions)

185089

Оставили в датасете только сматчившихся с wikidata исполнителей, выбрали случайно 0.125% примеров и удалили пользователей и песни со слишким маленьким числом взаимодействий. В итоге имеем выборку из 4 156 987 прослушываний.  

In [361]:
graph_dict = {}
song2albom = {}

In [362]:
with open('lastfm/events.csv', 'w', encoding='utf-8') as events_f, \
        open('lastfm/item_index2entity_id_rehashed.txt', 'w', encoding='utf-8') as mappings_f:
    for user_id, artist_id, albom_id, song_id in interactions:
        print(f'{user_id}\t{song_id}\t1', file=events_f)
        song_str = f'song#{song_id}'
        if song_str not in graph_dict:
            graph_dict[song_str] = len(graph_dict)
            print(f'{song_id}\t{graph_dict[song_str]}', file=mappings_f)

In [365]:
relations = set()

In [366]:
for user_id, artist_id, albom_id, song_id in interactions:
    if artist2wd[artist_id] not in graph_dict:
        graph_dict[artist2wd[artist_id]] = len(graph_dict)
    if f'albom#{albom_id}' not in graph_dict:
        graph_dict[f'albom#{albom_id}'] = len(graph_dict)
        
    relations.add((
        graph_dict[f'song#{song_id}'],
        'authorship',
        graph_dict[artist2wd[artist_id]]
    ))
    relations.add((
        graph_dict[artist2wd[artist_id]],
        'authorship_reversed',
        graph_dict[f'song#{song_id}']
    ))
    
    relations.add((
        graph_dict[f'song#{song_id}'],
        'part_of',
        graph_dict[f'albom#{albom_id}']
    ))
    relations.add((
        graph_dict[f'albom#{albom_id}'],
        'contains',
        graph_dict[f'song#{song_id}']
    ))
    
    relations.add((
        graph_dict[f'albom#{albom_id}'],
        'authorship',
        graph_dict[artist2wd[artist_id]]
    ))
    relations.add((
        graph_dict[artist2wd[artist_id]],
        'authorship_reversed',
        graph_dict[f'albom#{albom_id}']
    ))

In [367]:
for artist_id, data in datas.items():
    if data['item'] not in graph_dict:
        continue
    if 'country' in data and data['country']:
        if data['country'] not in graph_dict:
            graph_dict[data['country']] = len(graph_dict)
        relations.add((
            graph_dict[data['item']],
            'originalCountry',
            graph_dict[data['country']]
        ))
        relations.add((
            graph_dict[data['country']],
            'countryOf',
            graph_dict[data['item']]
        ))
    for genre in data['genres'].strip().split('|'):
        if genre not in graph_dict:
            graph_dict[genre] = len(graph_dict)
        relations.add((
            graph_dict[data['item']],
            'genre',
            graph_dict[genre]
        ))
        relations.add((
            graph_dict[genre],
            'genreOf',
            graph_dict[data['item']]
        ))
    for influenser in data['influensers'].strip().split('|'):
        if influenser not in graph_dict:
            graph_dict[influenser] = len(graph_dict)
        relations.add((
            graph_dict[data['item']],
            'influensedBy',
            graph_dict[influenser]
        ))
        relations.add((
            graph_dict[influenser],
            'influenseTo',
            graph_dict[data['item']]
        ))
    for award in data['awards'].strip().split('|'):
        if award not in graph_dict:
            graph_dict[award] = len(graph_dict)
        relations.add((
            graph_dict[data['item']],
            'awardedBy',
            graph_dict[award]
        ))  
        relations.add((
            graph_dict[award],
            'awardedTo',
            graph_dict[data['item']]
        ))  

In [368]:
len(relations)

177120

In [369]:
with open('lastfm/kg_rehashed.txt', 'w', encoding='utf-8') as f:
    for a, relation, b in relations:
        print(f'{a}\t{relation}\t{b}', file=f)